In [ ]:
#CatBoost Model
#from catboost import CatBoostClassifier, cv, Pool, metrics
#import hyperopt


In [ ]:
#categorical_features_indices = np.where(X.dtypes != float)[0]

#identifying best parameters for catboost
def hyperopt_objective(params):
    catboost_model = CatBoostClassifier(
        l2_leaf_reg=(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric=metrics.Accuracy(),
        random_seed=40,
        verbose=False,
        loss_function=metrics.Logloss(),
    )

    cv_data = cv(
        Pool(X_train, y_train, cat_features=categorical_features_indices),
        catboost_model.get_params(),
        logging_level='Silent',
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])

    return 1 - best_accuracy

In [ ]:
from numpy.random import default_rng

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space = params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=default_rng(123)
)

print(best)

In [ ]:
#getting cv with our best parameters
catboost_best_model = CatBoostClassifier(
    l2_leaf_reg=(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric=metrics.Accuracy(),
    random_seed=42,
    verbose=False,
    auto_class_weights="Balanced",
    loss_function=metrics.Logloss(),
)

cv_data = cv(Pool(X_train, y_train, cat_features=categorical_features_indices), catboost_best_model.get_params())

In [ ]:
#precise validation score
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))


# training model on the training dataset now
catboost_best_model.fit(X_train, y_train, cat_features=categorical_features_indices)


In [ ]:
#predicting performance on testing dataset
y_pred_catboost = catboost_best_model.predict(X_test)
y_pred_proba_catboost = catboost_best_model.predict_proba(X_test)[:,1]

#calculating accuracy
accuracy_catboost = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_catboost:.4f}")

#calculating AUC
auc_catboost = roc_auc_score(y_test, y_pred_proba_catboost)
print(f"ROC AUC: {auc_catboost:.4f}")

#classification report for catboost
print("\nClassification Report:\n", classification_report(y_test, y_pred_catboost))

#saving model
catboost_best_model.save_model('catboost_model.dump')

#loading model
#catboost_best_model.load_model('catboost_model.dump')

